In [1]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
cap = cv2.VideoCapture(1)

In [3]:
def get_frame_rgb(cap):
    _, frame = cap.read()
    frame = np.copy(frame)
    #plt.imshow(frame)
    target_width = 450
    target_height = frame.shape[1] * target_width / frame.shape[0]
    frame1 = cv2.resize(frame, (target_height, target_width))
    framergb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    return framergb

In [4]:
def get_mask(framergb):
    frame2 = cv2.cvtColor(framergb, cv2.COLOR_RGB2HSV)
    
    lower_red = np.array([160,50,50])
    upper_red = np.array([179,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(frame2, lower_red, upper_red)

    # Bitwise-AND mask and original image
    #res = cv2.bitwise_and(frame2, frame2, mask = mask)
    #plt.imshow(mask)
    return mask

In [5]:
def get_keypoints(mask):
    canvas = np.copy(framergb)

    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 100;
    params.maxThreshold = 100;

    # Filter by Area.
    params.filterByArea = False
    params.minArea = 1500

    # Filter by Circularity
    params.filterByCircularity = False
    params.minCircularity = 0.1

    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = False
    params.minInertiaRatio = 0.01

    detector = cv2.SimpleBlobDetector_create()

    # Detect blobs.
    keypoints = detector.detect(255-mask)

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(canvas, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    points = [kp.pt for kp in keypoints]

    #plt.imshow(im_with_keypoints)
    return np.array(points)

In [11]:
def draw_points(points, last_points, framergb):
    def coords_to_cv(p):
        return tuple([int(t) for t in p])
    def get_dist(p1, p2):
        return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2
    canvas = np.copy(framergb)
    assigned_indices = []
    distances = []
    for p in points:
        min_i = -1
        for i, p1 in enumerate(last_points):
            if min_i == -1:
                min_i = i
            dist = get_dist(last_points[i], p)
            min_dist = get_dist(last_points[min_i], p)
            if dist < min_dist:
                min_i = i
        distances.append(get_dist(last_points[min_i], p))
        assigned_indices.append(min_i)
        
    th = np.median(distances)
        
    for i in range(len(points)):
        p = points[i]
        dist = distances[i]
        p1 = last_points[assigned_indices[i]]
        if dist <= th:
            cv2.line(canvas, coords_to_cv(p1), coords_to_cv(p), (0, 255, 0))
            cv2.circle(canvas, coords_to_cv(p), 3, (0, 0, 255))
    return canvas

In [12]:
last_points = []
while(True):
    framergb = get_frame_rgb(cap)
    mask = get_mask(framergb)
    points = get_keypoints(mask)
    if len(last_points) > 0:
        canvas = draw_points(points, last_points, framergb)
        cv2.imshow('frame', canvas)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    last_points = np.copy(points)

/home/sergei/Documents/repos/git/EPFL/CHILI/Cellulo/qml-ar/notebooks/venv/local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sergei/Documents/repos/git/EPFL/CHILI/Cellulo/qml-ar/notebooks/venv/local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 